# 

In [4]:
# Standard library imports
import os
import shutil
import glob
from pathlib import Path
import logging


# Third-party library imports
import micom
from micom import Community
from cobra import Reaction, Metabolite, Model
from cobra.io import load_json_model, save_json_model, load_matlab_model, save_matlab_model, read_sbml_model, write_sbml_model
import cobra


In [5]:
# params
model_db_path = Path("./models/")       # Replace with your actual model directory path
adjusted_models_path = Path("./adjusted_models")  # Replace with your desired output directory path

# Define the medium composition
medium = {
    'EX_cpd00058_e0': 1000.0,   # Cu2+
    'EX_cpd00104_e0': 1000.0,   # Biotin
    'EX_cpd00063_e0': 1000.0,   # Ca2+
    'EX_cpd00099_e0': 1000.0,   # Cl-
    'EX_cpd00149_e0': 1000.0,   # Cobalt
    'EX_cpd10515_e0': 1000.0,   # Fe2+
    'EX_cpd10516_e0': 1000.0,   # Fe3+
    'EX_cpd00067_e0': 1000.0,   # H+
    'EX_cpd00001_e0': 1000.0,   # H2O
    'EX_cpd00205_e0': 1000.0,   # K+
    'EX_cpd00254_e0': 1000.0,   # Mg
    'EX_cpd00030_e0': 1000.0,   # Mn2+
    'EX_cpd11574_e0': 1000.0,   # Molybdate
    'EX_cpd00244_e0': 1000.0,   # Nickel
    'EX_cpd00209_e0': 1000.0,   # Nitrate
    'EX_cpd00009_e0': 1000.0,   # Phosphate
    'EX_cpd00048_e0': 1000.0,   # Sulfate
    'EX_cpd00305_e0': 1000.0,   # Thiamin
    'EX_cpd00034_e0': 1000.0,   # Zn2+
    'EX_cpd11632_e0': 1000.0,   # Photon
    'EX_cpd00011_e0': 1000.0,   # CO2
    'EX_cpd00007_e0': 1000.0,   # O2
    'EX_cpd00528_e0': 1000.0,   # N2
}

In [7]:
# ----------------------------
# Configure Logging
# ----------------------------

# First, get the root logger
logger = logging.getLogger()
logger.setLevel(logging.INFO)  # Set the desired logging level

# Remove any existing handlers to prevent duplicate logs
if logger.hasHandlers():
    logger.handlers.clear()

# Create a file handler to save logs to a file
file_handler = logging.FileHandler('model_processing.log')
file_handler.setLevel(logging.INFO)

# Create a stream handler to print logs to the Jupyter cell
stream_handler = logging.StreamHandler()
stream_handler.setLevel(logging.INFO)

# Define a common formatter
formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')

# Assign the formatter to both handlers
file_handler.setFormatter(formatter)
stream_handler.setFormatter(formatter)

# Add both handlers to the logger
logger.addHandler(file_handler)
logger.addHandler(stream_handler)



def adjust_medium(model, medium):
    """
    Adjust the medium for the given model based on available exchange reactions.
    """
    adjusted_medium = {}
    available_exchanges = {rxn.id for rxn in model.exchanges}
    for rxn_id, value in medium.items():
        if rxn_id in available_exchanges:
            adjusted_medium[rxn_id] = value
        else:
            logging.warning(f"Exchange reaction {rxn_id} not found in model {model.id}")
    return adjusted_medium

def add_exchange_reaction(model, metabolite_id_base, lower_bound=-1000, upper_bound=1000):
    """
    Adds an exchange reaction for a metabolite, creating external and transport reactions if necessary.
    """
    # Build metabolite IDs
    internal_met_id = f"M_{metabolite_id_base}_c0"
    external_met_id = f"M_{metabolite_id_base}_e0"
    exchange_rxn_id = f"EX_M_{metabolite_id_base}_e0"
    transport_rxn_id = f"TRANS_M_{metabolite_id_base}"
    
    # Check if the internal metabolite exists
    try:
        internal_met = model.metabolites.get_by_id(internal_met_id)
    except KeyError:
        logging.error(f"Internal metabolite {internal_met_id} not found in model {model.id}. Cannot proceed.")
        return
    
    # Check if the external metabolite exists; if not, create it
    external_met = model.metabolites.get_by_id(external_met_id, None)
    if external_met is None:
        external_met = Metabolite(
            id=external_met_id,
            formula=internal_met.formula,
            name=internal_met.name,
            compartment='e0'
        )
        model.add_metabolites([external_met])
        logging.info(f"Added external metabolite {external_met_id} to model {model.id}")
    
    # Add the transport reaction if it doesn't exist
    if transport_rxn_id not in model.reactions:
        transport_rxn = Reaction(id=transport_rxn_id)
        transport_rxn.name = f"Transport of {internal_met.name}"
        transport_rxn.lower_bound = -1000
        transport_rxn.upper_bound = 1000
        transport_rxn.add_metabolites({internal_met: -1, external_met: 1})
        model.add_reactions([transport_rxn])
        logging.info(f"Added transport reaction {transport_rxn_id} to model {model.id}")
    
    # Add the exchange reaction if it doesn't exist
    if exchange_rxn_id not in model.reactions:
        exchange_rxn = Reaction(id=exchange_rxn_id)
        exchange_rxn.name = f"Exchange of {external_met.name}"
        exchange_rxn.lower_bound = lower_bound
        exchange_rxn.upper_bound = upper_bound
        exchange_rxn.add_metabolites({external_met: -1})
        model.add_reactions([exchange_rxn])
        logging.info(f"Added exchange reaction {exchange_rxn_id} to model {model.id}")
    else:
        logging.info(f"Exchange reaction {exchange_rxn_id} already exists in model {model.id}")

def add_all_exchange_reactions(model, medium):
    """
    Add exchange reactions for all components defined in the medium.
    """
    for rxn_id in medium.keys():
        if rxn_id not in model.reactions:
            # Infer metabolite ID by removing 'EX_' prefix
            if rxn_id.startswith('EX_'):
                metabolite_id = rxn_id[3:]
                if metabolite_id in model.metabolites:
                    add_exchange_reaction(model, metabolite_id, reaction_id=rxn_id)
                else:
                    logging.warning(f"Metabolite {metabolite_id} corresponding to {rxn_id} not found in model {model.id}. Exchange reaction {rxn_id} not added.")
            else:
                logging.warning(f"Reaction ID {rxn_id} does not follow the 'EX_' prefix convention. Skipping.")

def process_model(model_path, output_dir, medium):
    """
    Load a model, adjust it by adding exchange reactions for all medium components,
    set the medium, optimize the model, and save the adjusted model to the output directory.
    """
    try:
        # Load the model using micom.util.load_model (keeping it as is)
        model = micom.util.load_model(str(model_path))
        logging.info(f"\nProcessing model: {model.id}")

        # Add exchange reactions for all medium components
        add_all_exchange_reactions(model, medium)

        # Adjust the medium
        adjusted_medium = adjust_medium(model, medium)
        model.medium = adjusted_medium

        # Optional: Optimize the model to ensure it can grow
        solution = model.optimize()
        logging.info(f"Model {model.id} growth rate after adjustment: {solution.objective_value}")

        # Save the adjusted model using the existing save function
        output_model_path = output_dir / model_path.name
        write_sbml_model(model, output_model_path)  # Keeping the saving function as is
        logging.info(f"Saved adjusted model to {output_model_path}")

    except Exception as e:
        logging.error(f"Failed to process model {model_path.name}: {e}")


"""
Main function to process all models in the specified directory.
"""
# Iterate through all files in the model_db_path
for model_file in model_db_path.iterdir():
    # Check if the file is an XML file (common format for metabolic models)
    if model_file.suffix.lower() in ['.xml']:
        process_model(model_file, adjusted_models_path, medium)
    else:
        logging.warning(f"Skipping unsupported file format: {model_file.name}")

logging.info("\nAll models have been processed.")

# ----------------------------
# Copy Manifest File to the Output Directory
# ----------------------------

# Define the manifest file name (adjust if your manifest file has a different name)
manifest_filename = "manifest.csv"  # Replace with the actual manifest file name if different

# Define the source and destination paths for the manifest file
source_manifest = model_db_path / manifest_filename
destination_manifest = adjusted_models_path / manifest_filename

# Check if the manifest file exists in the source directory
if source_manifest.exists():
    try:
        shutil.copy2(source_manifest, destination_manifest)
        logging.info(f"Copied manifest file to {destination_manifest}")
    except Exception as e:
        logging.error(f"Failed to copy manifest file: {e}")
else:
    logging.warning(f"Manifest file {manifest_filename} not found in {model_db_path}. No manifest copied.")


2024-11-11 10:56:30,033 - INFO - reading model from models/bin.5.xml
2024-11-11 10:56:33,526 - INFO - 
Processing model: bin_5
2024-11-11 10:56:33,528 - WARNING - Metabolite cpd00104_e0 corresponding to EX_cpd00104_e0 not found in model bin_5. Exchange reaction EX_cpd00104_e0 not added.
2024-11-11 10:56:33,536 - INFO - Compartment `e0` sounds like an external compartment. Using this one without counting boundary reactions.
2024-11-11 10:56:33,545 - WARNING - Exchange reaction EX_cpd00104_e0 not found in model bin_5
2024-11-11 10:56:33,550 - INFO - Compartment `e0` sounds like an external compartment. Using this one without counting boundary reactions.
2024-11-11 10:56:33,622 - INFO - Model bin_5 growth rate after adjustment: 0.0
2024-11-11 10:56:35,698 - INFO - Saved adjusted model to adjusted_models/bin.5.xml
2024-11-11 10:56:35,722 - INFO - reading model from models/bin.6.xml
2024-11-11 10:56:38,971 - INFO - 
Processing model: bin_6
2024-11-11 10:56:38,973 - WARNING - Metabolite cpd0

In [8]:
# Combine Individual Models into a Community Model
import pandas as pd
# Create the taxonomy DataFrame
taxonomy = pd.DataFrame({
    'id': ['bin.5','bin.6'],
    'abundance': [0.5,0.5],
    'phylum': ['Firmicutes','Cyanobacteria']
})

# Specify the directory containing your model files

# Create the community model
community = Community (
    taxonomy,
     model_db=str(adjusted_models_path)
)

2024-11-11 10:56:52,430 - INFO - building new micom model None.
2024-11-11 10:56:52,432 - INFO - using the cplex solver.
2024-11-11 10:56:52,439 - INFO - 0 individuals with abundances below threshold
2024-11-11 10:56:52,449 - INFO - Matched 100% of total abundance in model DB.


Output()

2024-11-11 10:56:52,506 - INFO - reading model from adjusted_models/bin.5.xml
2024-11-11 10:56:55,876 - INFO - converting IDs for bin_5
2024-11-11 10:56:55,885 - INFO - Compartment `e0` sounds like an external compartment. Using this one without counting boundary reactions.
2024-11-11 10:56:55,886 - INFO - Identified e0 as the external compartment for bin_5. If that is wrong you may be in trouble...
2024-11-11 10:57:01,424 - INFO - adding reactions for bin_5 to community
2024-11-11 10:57:02,162 - INFO - adding metabolite cpd00001_m to external medium
2024-11-11 10:57:02,165 - INFO - adding metabolite cpd00418_m to external medium
2024-11-11 10:57:02,167 - INFO - adding metabolite cpd00075_m to external medium
2024-11-11 10:57:02,169 - INFO - adding metabolite cpd00009_m to external medium
2024-11-11 10:57:02,171 - INFO - adding metabolite cpd00023_m to external medium
2024-11-11 10:57:02,177 - INFO - adding metabolite cpd00027_m to external medium
2024-11-11 10:57:02,179 - INFO - addin